In [44]:
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.common.by import By 
import time
import lyricsgenius

In [2]:
def init_driver():
    driver = webdriver.Firefox()
    return driver

In [ ]:
driver = init_driver()

### Getting all artists from canzoni.it

In [33]:
def get_artists(driver):
    artists_list = []
    web_link = 'https://www.canzoni.it/artisti/cantanti/italiani/'
    for char in range(97, 123):
        driver.get(web_link+chr(char))
        lists = driver.find_elements(By.CSS_SELECTOR, 'ul.ul_freccia')
        for list in lists:
            els = list.find_elements(By.CSS_SELECTOR, 'li a')   
            for el in els:
                artists_list.append(el.text)    
    return artists_list

In [34]:
artists_list = get_artists(driver)

In [36]:
with open('artists_list.txt', 'w') as artists_file:
    for artist in artists_list:
        artists_file.write(artist+'\n')

### Getting each artist albums and songs from Genius

In [37]:
from genius import Genius
g = Genius(access_token='JM0ShY3X3lUnih5cEfdJr60wQZ6Pgh4GerRQb8pOqI4yNlFJaUE8v0u0mm9hU_zA')

In [279]:
def fill_artists_list():
    artists_list = []
    with open('artists_list.txt', 'r', encoding='utf-8') as file_artists:
        for line in file_artists:
            artists_list.append(line.strip())
    return artists_list

In [478]:
def get_songs_dict(artists_list, dict_songs, not_found):
    for artist in artists_list:
        try:
            gen_artist = g.search_artist(artist)
            print(artist)
            for song in gen_artist.songs:
                if song not in dict_songs:
                    song_info = {}
                    song_info['Title'] = song.title
                    song_info['Artist'] = song.artist.name
                    song_info['Features'] = song.features
                    song_info['Album'] = song.album
                    song_info['Release_Date'] = song.release_date
                    song_info['Producers'] = song.producers
                    song_info['Writers'] = song.writers
                    song_info['Cover_Img'] = song.song_art_image_url
                    dict_songs[song] = song_info
                    print(song)
                    
        except Exception as exc:
            print(exc)
            print('Not Found', artist)
            not_found.append(artist)
            continue
    return dict_songs, not_found

In [401]:
artists_list = fill_artists_list()

In [ ]:
songs_dict, not_found = get_songs_dict(artists_list, songs_dict, not_found)

In [482]:
pd.DataFrame(songs_dict).T.to_csv('songs_dict.csv')

In [360]:
def write_not_found(not_found):
    with open('not_found.txt', 'w') as f:
        for name in not_found:
            f.write(name+'\n')

In [431]:
write_not_found(not_found)